In [1]:
import kotlinx.coroutines.Dispatchers
import kotlinx.coroutines.async
import kotlinx.coroutines.awaitAll
import kotlinx.coroutines.runBlocking
import com.google.common.hash.Hashing
import io.github.cdimascio.dotenv.Dotenv
import java.nio.file.Paths

%useLatestDescriptors
%use lets-plot

class Main

val dotenv = Dotenv.load()
val dataDir = dotenv.get("DATA_DIR").let { Paths.get(it).toFile() }.also { it.mkdirs() }
dataDir

/home/j.zeidler@GDAAG.DE/private/ba3/bachelor-thesis/data

In [2]:
import org.bson.BsonDocument
import org.litote.kmongo.*

val client = KMongo.createClient("mongodb://localhost:42692/")
val db = client.getDatabase("s5_snyk_libio")
val vulnCollection = db.getCollection<BsonDocument>("mergedVuln")

In [3]:
import com.mongodb.client.*

private class MongoCursorIterable<T>(private val cursor: MongoCursor<T>) : MongoCursor<T> by cursor, Iterable<T> {

    override fun iterator(): Iterator<T> = cursor
}

private fun <T> MongoIterable<T>.kCursor(): MongoCursorIterable<T> = MongoCursorIterable(iterator())

fun <T, R> MongoIterable<T>.useCursor(block: (Iterable<T>) -> R): R {
    return kCursor().use(block)
}

In [4]:
import org.bson.BsonNull
import org.bson.BsonString
import org.jetbrains.kotlinx.dataframe.math.mean
import org.jetbrains.kotlinx.dataframe.math.median

val vulnGavToClasses = vulnCollection.find().useCursor { blk ->
    blk.map {
        val gav = it["vuln_gav"]!!.asString().value
        val vulnClasses = it["vuln_classes"]!!.asArray().map { it.asString().value }.toSet()

        if (vulnClasses.count() == 0) throw Exception("no vuln class (record should have been ommitted previously)")

        gav to vulnClasses
    }
}.groupBy { it.first }.map { g -> g.key to g.value.flatMap { it -> it.second }.toSet() }.toMap()

println(vulnGavToClasses.count())
println(vulnGavToClasses.map { it.value.size }.minOrNull())
println(vulnGavToClasses.map { it.value.size }.maxOrNull())
println(vulnGavToClasses.map { it.value.size }.average())
println(vulnGavToClasses.map { it.value.size }.median())

3809
1
51
3.549750590706222
2


In [5]:
val vulnCveGavToClasses = vulnCollection.find().useCursor { blk ->
    blk.map {
        val gav = it["vuln_gav"]!!.asString().value
        val cve = it["cve_ref"]!!.asString().value.let { c -> if (c.isBlank()) it["snyk_url"]!!.asString().value else c }
        val vulnClasses = it["vuln_classes"]!!.asArray().map { it.asString().value }.toSet()

        if (vulnClasses.count() == 0) throw Exception("no vuln class (record should have been ommitted previously)")

        (cve to gav) to vulnClasses
    }
}.groupBy { it.first }.map { g -> g.key to g.value.flatMap { it -> it.second }.toSet() }.toMap()
    
println(vulnCveGavToClasses.count())
println(vulnCveGavToClasses.map { it.value.size }.minOrNull())
println(vulnCveGavToClasses.map { it.value.size }.maxOrNull())
println(vulnCveGavToClasses.map { it.value.size }.average())
println(vulnCveGavToClasses.map { it.value.size }.median())

7618
1
51
2.0490942504594383
1


In [6]:
val depGraphCacheDir = dataDir.resolve("interim/depGraphCache")

val gavToDepGraphCache = vulnGavToClasses
    .map {
        val cachePath = depGraphCacheDir.resolve("${it.key}.vertices.tsv.zip")
        if (!cachePath.isFile()) throw Exception("cache file not found")
        
        it.key to cachePath
    }
    .toMap()
gavToDepGraphCache.count()

3809

In [7]:
import common.DefaultGraph
import io.github.classgraph.ClassInfoList
import java.io.File
import java.net.URLClassLoader
import org.jgrapht.Graph
import org.jgrapht.graph.DefaultEdge
import org.jgrapht.graph.builder.GraphTypeBuilder

fun loadDepGraph(gav: String): DefaultGraph = 
    scripts.exportDepGraphs.loadDepGraphFromCache(gav)

fun loadClassListInfo(gav: String): ClassInfoList =
    scripts.exportDepGraphs.loadClassInfoListFromCache(gav)

fun loadVertexInfo(gav: String) =
    scripts.exportDepGraphs.loadVertexInfo(gav)

### how much of the public API is reaching vulns?

#### first we count if it hits "any" vuln class:

In [8]:
import com.google.common.collect.Queues

/**
 * returns the depths of each visited node from the starting node
 */
fun bfsOnDepGraph(graph: DefaultGraph, startNode: String): Map<String, Int> {
    val expanded = mutableSetOf<String>()
    val depthMap = mutableMapOf<String, Int>()
    val queue = Queues.newArrayDeque<String>()
    queue.add(startNode)
    depthMap[startNode] = 0
    while (!queue.isEmpty()) {
        val node = queue.pop()
        expanded.add(node)
        val parentDepth = depthMap[node]!! 
        graph
            .outgoingEdgesOf(node)
            .map { e -> graph.getEdgeTarget(e) }
            .filterNot { expanded.contains(it) }
            .forEach { v ->
                if (depthMap.containsKey(v)) {
                    depthMap[v] = min(depthMap[v]!!, parentDepth + 1)
                } else {
                    depthMap[v] = parentDepth + 1;
                }
                queue.add(v)
            }
    }
    return depthMap
}

In [9]:
import common.DefaultGraph
import io.github.classgraph.ClassGraph
import io.github.classgraph.ClassInfo
import io.github.classgraph.ClassInfoList
import kotlinx.coroutines.async
import org.jgrapht.alg.shortestpath.FloydWarshallShortestPaths
import org.jgrapht.alg.shortestpath.GraphMeasurer
import org.jgrapht.graph.EdgeReversedGraph
import org.jgrapht.traverse.BreadthFirstIterator
import java.lang.reflect.Modifier
import kotlin.Exception

data class VulnAnalysisInfo(
    val classCount: Int,
    val publicClassCount: Int,
    val vulnClassCount: Int,
    val vulnPublicClassCount: Int,
    val averageDistToVuln: Double,
    val minDistToVuln: Int,
    val maxDistToVuln: Int,
    val medianDistToVuln: Int,
    // val publicClassesWithPublicApiCount: Int,
    // val depGraph: DefaultGraph,
) {
    val apiSurfaceRatio = publicClassCount.toDouble() / classCount.toDouble()
    val vulnApiSurfaceRatio = vulnPublicClassCount.toDouble() / publicClassCount.toDouble()
    val vulnRatio = vulnClassCount.toDouble() / classCount.toDouble()
}

System.gc()

@kotlinx.coroutines.ExperimentalCoroutinesApi fun _vulnInfo() =
    Dispatchers.IO.limitedParallelism(32).let { dispatcher ->
        vulnCveGavToClasses.asSequence().windowed(512, 512, true).flatMapIndexed { batch, w ->
            runBlocking {
                System.gc()
                println("processing batch $batch")
                w.map {
                    async(dispatcher) {
                        val gav = it.key.second
                        val vulnClasses = it.value

                        val depGraph = loadDepGraph(gav)
                        
                        val vertexInfo = loadVertexInfo(gav)

                        val publicClasses =
                            depGraph.vertexSet().filter { v -> Modifier.isPublic(vertexInfo[v]!!) }

                        if (publicClasses.count() > depGraph.vertexSet().count()) {
                            error("#public classes should be <= #classes")
                        }

                        if (publicClasses.count() == 0) {
                            return@async Result.failure(Exception("no public class in $gav"))
                        }

                        val vulnClassesInJar = vulnClasses.intersect(depGraph.vertexSet())
                        if (vulnClassesInJar.size != vulnClasses.size) {
                            error("vuln class number mismatch ${gav}\n***\n$vulnClassesInJar\n***\n$vulnClasses")
                        }

                        val distToNearestVuln = mutableMapOf<String, Int>()
                        vulnClassesInJar.map {
                            // BreadthFirstIterator(EdgeReversedGraph(depGraph), it).asSequence().toSet()
                            bfsOnDepGraph(EdgeReversedGraph(depGraph), it)
                        }.forEach { m ->
                            m.forEach { v, d -> 
                                if (distToNearestVuln.containsKey(v)) {
                                    distToNearestVuln[v] = min(distToNearestVuln[v]!!, d)
                                } else {
                                    distToNearestVuln[v] = d
                                }
                            }
                        }
                        
                        val allVulnClasses = distToNearestVuln.keys.toSet()

                        val vulnPublicClasses = publicClasses.intersect(allVulnClasses)

                        val vulnPublicClassRatio =
                            vulnPublicClasses.count().toDouble() / publicClasses.count().toDouble()

                        if (vulnPublicClassRatio > 1) {
                            error("ratio should be <= 1")
                        }
                        
                        val averageDistToVuln = distToNearestVuln.values.average()
                        val minDistToVuln = distToNearestVuln.values.min()
                        val maxDistToVuln = distToNearestVuln.values.max()
                        val medianDistToVuln = distToNearestVuln.values.median()
                        
                        // val classes = loadClassListInfo(gav)
                        // val classToClassInfo = classes.filter { it.name in depGraph.vertexSet() }.map { it.name to it }.toMap()

                        // val publicClassesWithPublicApi =
                        //     publicClasses
                        //         .filter { c -> classToClassInfo[c]!!.let { it.methodInfo.any { it.isPublic } || it.fieldInfo.any { it.isPublic } } }
                        //         .toSet()

                        val res = VulnAnalysisInfo(
                            classCount = depGraph.vertexSet().count(),
                            publicClassCount = publicClasses.count(),
                            vulnClassCount = allVulnClasses.count(),
                            vulnPublicClassCount = vulnPublicClasses.count(),
                            averageDistToVuln = averageDistToVuln,
                            minDistToVuln = minDistToVuln,
                            maxDistToVuln = maxDistToVuln,
                            medianDistToVuln = medianDistToVuln,
                            // publicClassesWithPublicApiCount = publicClassesWithPublicApi.count()
                            // depGraph = depGraph,
                        )

                        return@async Result.success(res)
                    }
                }.toList().awaitAll().toList()
            }.toList().filter { it.isSuccess }.map { it.getOrNull()!! }.toList()
        }.toList()
    }

val vulnInfo = _vulnInfo()
vulnInfo.count()

processing batch 0
processing batch 1
processing batch 2
processing batch 3
processing batch 4
processing batch 5
processing batch 6
processing batch 7
processing batch 8
processing batch 9
processing batch 10
processing batch 11
processing batch 12
processing batch 13
processing batch 14


7618

In [10]:
System.gc()

In [11]:
fun Double.format(digits: Int) =
    "%.${digits}f".format(this)

fun Float.format(digits: Int) =
    "%.${digits}f".format(this)

In [12]:
println("classCount median: ${vulnInfo.map { it.classCount }.median()} avg: ${vulnInfo.map { it.classCount }.average().format(2)}")
println("publicClassCount median: ${vulnInfo.map { it.publicClassCount }.median()} avg: ${vulnInfo.map { it.publicClassCount }.average().format(2)}")
// DISPLAY("publicClassesWithPublicApiCount: ${vulnInfo.map { it.publicClassesWithPublicApiCount }.median()}")
println("vulnClassCount median: ${vulnInfo.map { it.vulnClassCount }.median()} avg: ${vulnInfo.map { it.vulnClassCount }.average().format(2)}")
println("vulnPublicClassCount median: ${vulnInfo.map { it.vulnPublicClassCount }.median()} avg: ${vulnInfo.map { it.vulnPublicClassCount }.average().format(2)}")
println("apiSurfaceRatio median: ${vulnInfo.map { it.apiSurfaceRatio * 100 }.median().format(2)} avg: ${vulnInfo.map { it.apiSurfaceRatio * 100 }.average().format(2)}")
// DISPLAY("publicClassesWithPublicApiRatio: ${vulnInfo.map { it.publicClassesWithPublicApiCount.toDouble() / it.classCount.toDouble() }.map { it * 100 }.median()}")
println("vulnApiSurfaceRatio median: ${vulnInfo.map { it.vulnApiSurfaceRatio * 100 }.median().format(2)} avg: ${vulnInfo.map { it.vulnApiSurfaceRatio * 100 }.average().format(2)}")

classCount median: 293 avg: 567.94
publicClassCount median: 277 avg: 522.39
vulnClassCount median: 14 avg: 182.61
vulnPublicClassCount median: 13 avg: 172.35
apiSurfaceRatio median: 94.27 avg: 92.06
vulnApiSurfaceRatio median: 10.00 avg: 26.26


In [13]:
val p =
    letsPlot(mapOf(
        "number of classes" to vulnInfo.map { it.classCount.coerceAtMost(2800) })
    ) +
            geomHistogram(alpha = .3, binWidth = 200, center = 100) { x = "number of classes"; } +
            xlim(limits = Pair(0, 3000)) +
            ggsize(1800, 800)
p

-100 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 
 
 700 
 
 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 
 
 900 
 
 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 
 
 1,100 
 
 
 
 
 
 
 
 
 1,200 
 
 
 
 
 
 
 
 
 1,300 
 
 
 
 
 
 
 
 
 1,400 
 
 
 
 
 
 
 
 
 1,500 
 
 
 
 
 
 
 
 
 1,600 
 
 
 
 
 
 
 
 
 1,700 
 
 
 
 
 
 
 
 
 1,800 
 
 
 
 
 
 
 
 
 1,900 
 
 
 
 
 
 
 
 
 2,000 
 
 
 
 
 
 
 
 
 2,100 
 
 
 
 
 
 
 
 
 2,200 
 
 
 
 
 
 
 
 
 2,300 
 
 
 
 
 
 
 
 
 2,400 
 
 
 
 
 
 
 
 
 2,500 
 
 
 
 
 
 
 
 
 2,600 
 
 
 
 
 
 
 
 
 2,700 
 
 
 
 
 
 
 
 
 2,800 
 
 
 
 
 
 
 
 
 2,900 
 
 
 
 
 
 
 
 
 3,000 
 
 
 
 
 
 
 
 
 3,100 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 1,200 
 
 
 
 
 
 
 1,400 
 
 
 
 
 
 
 1,600 
 
 
 
 
 
 
 1,800 
 
 
 
 
 
 
 2,000 
 
 
 
 
 
 
 2,200 
 
 
 
 
 
 
 2,400 
 
 
 
 
 
 
 2,600 
 
 
 
 
 
 
 2,800 
 
 
 
 
 
 
 3,000 
 
 
 
 
 
 
 
 count 
 
 
 
 
 number of classes

In [14]:
val p =
    letsPlot(mapOf(
        "number of public classes" to vulnInfo.map { it.publicClassCount.coerceAtMost(2800) })
    ) +
            geomHistogram(alpha = .3, binWidth = 200, center = 100) { x = "number of public classes"; } +
            xlim(limits = Pair(0, 3000)) +
            ggsize(1800, 800)
p

-100 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 
 
 700 
 
 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 
 
 900 
 
 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 
 
 1,100 
 
 
 
 
 
 
 
 
 1,200 
 
 
 
 
 
 
 
 
 1,300 
 
 
 
 
 
 
 
 
 1,400 
 
 
 
 
 
 
 
 
 1,500 
 
 
 
 
 
 
 
 
 1,600 
 
 
 
 
 
 
 
 
 1,700 
 
 
 
 
 
 
 
 
 1,800 
 
 
 
 
 
 
 
 
 1,900 
 
 
 
 
 
 
 
 
 2,000 
 
 
 
 
 
 
 
 
 2,100 
 
 
 
 
 
 
 
 
 2,200 
 
 
 
 
 
 
 
 
 2,300 
 
 
 
 
 
 
 
 
 2,400 
 
 
 
 
 
 
 
 
 2,500 
 
 
 
 
 
 
 
 
 2,600 
 
 
 
 
 
 
 
 
 2,700 
 
 
 
 
 
 
 
 
 2,800 
 
 
 
 
 
 
 
 
 2,900 
 
 
 
 
 
 
 
 
 3,000 
 
 
 
 
 
 
 
 
 3,100 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 1,200 
 
 
 
 
 
 
 1,400 
 
 
 
 
 
 
 1,600 
 
 
 
 
 
 
 1,800 
 
 
 
 
 
 
 2,000 
 
 
 
 
 
 
 2,200 
 
 
 
 
 
 
 2,400 
 
 
 
 
 
 
 2,600 
 
 
 
 
 
 
 2,800 
 
 
 
 
 
 
 3,000 
 
 
 
 
 
 
 3,200 
 
 
 
 
 
 
 
 count 
 
 
 
 
 number of public classes

In [15]:
val p =
    letsPlot(mapOf(
        "number of vulnerable classes" to vulnInfo.map { it.vulnClassCount.coerceAtMost(2800) })
    ) +
            geomHistogram(alpha = .3, binWidth = 200, center = 100) { x = "number of vulnerable classes"; } +
            xlim(limits = Pair(0, 3000)) +
            ggsize(1800, 800)
p

-100 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 
 
 700 
 
 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 
 
 900 
 
 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 
 
 1,100 
 
 
 
 
 
 
 
 
 1,200 
 
 
 
 
 
 
 
 
 1,300 
 
 
 
 
 
 
 
 
 1,400 
 
 
 
 
 
 
 
 
 1,500 
 
 
 
 
 
 
 
 
 1,600 
 
 
 
 
 
 
 
 
 1,700 
 
 
 
 
 
 
 
 
 1,800 
 
 
 
 
 
 
 
 
 1,900 
 
 
 
 
 
 
 
 
 2,000 
 
 
 
 
 
 
 
 
 2,100 
 
 
 
 
 
 
 
 
 2,200 
 
 
 
 
 
 
 
 
 2,300 
 
 
 
 
 
 
 
 
 2,400 
 
 
 
 
 
 
 
 
 2,500 
 
 
 
 
 
 
 
 
 2,600 
 
 
 
 
 
 
 
 
 2,700 
 
 
 
 
 
 
 
 
 2,800 
 
 
 
 
 
 
 
 
 2,900 
 
 
 
 
 
 
 
 
 3,000 
 
 
 
 
 
 
 
 
 3,100 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 1,500 
 
 
 
 
 
 
 2,000 
 
 
 
 
 
 
 2,500 
 
 
 
 
 
 
 3,000 
 
 
 
 
 
 
 3,500 
 
 
 
 
 
 
 4,000 
 
 
 
 
 
 
 4,500 
 
 
 
 
 
 
 5,000 
 
 
 
 
 
 
 5,500 
 
 
 
 
 
 
 6,000 
 
 
 
 
 
 
 
 count 
 
 
 
 
 number of vulnerable classes

In [16]:
val p =
    letsPlot(mapOf(
        "number of vuln public classes" to vulnInfo.map { it.vulnPublicClassCount.coerceAtMost(2800) })
    ) +
            geomHistogram(alpha = .3, binWidth = 200, center = 100) { x = "number of vuln public classes"; } +
            xlim(limits = Pair(0, 3000)) +
            ggsize(1800, 800)
p

-100 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 
 
 700 
 
 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 
 
 900 
 
 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 
 
 1,100 
 
 
 
 
 
 
 
 
 1,200 
 
 
 
 
 
 
 
 
 1,300 
 
 
 
 
 
 
 
 
 1,400 
 
 
 
 
 
 
 
 
 1,500 
 
 
 
 
 
 
 
 
 1,600 
 
 
 
 
 
 
 
 
 1,700 
 
 
 
 
 
 
 
 
 1,800 
 
 
 
 
 
 
 
 
 1,900 
 
 
 
 
 
 
 
 
 2,000 
 
 
 
 
 
 
 
 
 2,100 
 
 
 
 
 
 
 
 
 2,200 
 
 
 
 
 
 
 
 
 2,300 
 
 
 
 
 
 
 
 
 2,400 
 
 
 
 
 
 
 
 
 2,500 
 
 
 
 
 
 
 
 
 2,600 
 
 
 
 
 
 
 
 
 2,700 
 
 
 
 
 
 
 
 
 2,800 
 
 
 
 
 
 
 
 
 2,900 
 
 
 
 
 
 
 
 
 3,000 
 
 
 
 
 
 
 
 
 3,100 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 1,500 
 
 
 
 
 
 
 2,000 
 
 
 
 
 
 
 2,500 
 
 
 
 
 
 
 3,000 
 
 
 
 
 
 
 3,500 
 
 
 
 
 
 
 4,000 
 
 
 
 
 
 
 4,500 
 
 
 
 
 
 
 5,000 
 
 
 
 
 
 
 5,500 
 
 
 
 
 
 
 6,000 
 
 
 
 
 
 
 
 count 
 
 
 
 
 number of vuln public classes

In [17]:
val p =
    letsPlot(mapOf(
        "apiSurfaceRatio" to vulnInfo.map { it.apiSurfaceRatio * 100 })
    ) +
            geomHistogram(alpha = .3, binWidth = 5, center = 2.5) { x = "apiSurfaceRatio"; } +
            xlim(limits = Pair(0, 100)) +
            ggsize(1800, 800)
p

-5 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 
 
 15 
 
 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 
 
 25 
 
 
 
 
 
 
 
 
 30 
 
 
 
 
 
 
 
 
 35 
 
 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 
 
 45 
 
 
 
 
 
 
 
 
 50 
 
 
 
 
 
 
 
 
 55 
 
 
 
 
 
 
 
 
 60 
 
 
 
 
 
 
 
 
 65 
 
 
 
 
 
 
 
 
 70 
 
 
 
 
 
 
 
 
 75 
 
 
 
 
 
 
 
 
 80 
 
 
 
 
 
 
 
 
 85 
 
 
 
 
 
 
 
 
 90 
 
 
 
 
 
 
 
 
 95 
 
 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 
 
 105 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 1,200 
 
 
 
 
 
 
 1,400 
 
 
 
 
 
 
 1,600 
 
 
 
 
 
 
 1,800 
 
 
 
 
 
 
 2,000 
 
 
 
 
 
 
 2,200 
 
 
 
 
 
 
 2,400 
 
 
 
 
 
 
 2,600 
 
 
 
 
 
 
 
 count 
 
 
 
 
 apiSurfaceRatio

In [18]:
val p =
    letsPlot(mapOf(
        "vulnApiSurfaceRatio" to vulnInfo.map { it.vulnApiSurfaceRatio * 100 })
    ) +
            geomHistogram(alpha = .3, binWidth = 5, center = 2.5) { x = "vulnApiSurfaceRatio"; } +
            xlim(limits = Pair(0, 100)) +
            ggsize(1800, 800)
p

-5 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 
 
 15 
 
 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 
 
 25 
 
 
 
 
 
 
 
 
 30 
 
 
 
 
 
 
 
 
 35 
 
 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 
 
 45 
 
 
 
 
 
 
 
 
 50 
 
 
 
 
 
 
 
 
 55 
 
 
 
 
 
 
 
 
 60 
 
 
 
 
 
 
 
 
 65 
 
 
 
 
 
 
 
 
 70 
 
 
 
 
 
 
 
 
 75 
 
 
 
 
 
 
 
 
 80 
 
 
 
 
 
 
 
 
 85 
 
 
 
 
 
 
 
 
 90 
 
 
 
 
 
 
 
 
 95 
 
 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 
 
 105 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 1,200 
 
 
 
 
 
 
 1,400 
 
 
 
 
 
 
 1,600 
 
 
 
 
 
 
 1,800 
 
 
 
 
 
 
 2,000 
 
 
 
 
 
 
 2,200 
 
 
 
 
 
 
 2,400 
 
 
 
 
 
 
 2,600 
 
 
 
 
 
 
 2,800 
 
 
 
 
 
 
 3,000 
 
 
 
 
 
 
 3,200 
 
 
 
 
 
 
 
 count 
 
 
 
 
 vulnApiSurfaceRatio

what is the depth of vulnerability from public classes?

In [19]:
println("medianDistanceToVuln median: ${vulnInfo.map { it.medianDistToVuln }.median()} avg: ${vulnInfo.map { it.medianDistToVuln }.average().format(2)}")
println("averageDistanceToVuln median: ${vulnInfo.map { it.averageDistToVuln }.median()} avg: ${vulnInfo.map { it.averageDistToVuln }.average().format(2)}")
println("minDistanceToVuln median: ${vulnInfo.map { it.minDistToVuln }.median()} avg: ${vulnInfo.map { it.minDistToVuln }.average().format(2)}")
println("maxDistanceToVuln median: ${vulnInfo.map { it.maxDistToVuln }.median()} avg: ${vulnInfo.map { it.maxDistToVuln }.average().format(2)}")

medianDistanceToVuln median: 2 avg: 2.01
averageDistanceToVuln median: 1.6 avg: 2.00
minDistanceToVuln median: 0 avg: 0.00
maxDistanceToVuln median: 3 avg: 3.95


In [20]:
val p =
    letsPlot(mapOf(
        "medianDistanceToVuln" to vulnInfo.map { it.medianDistToVuln })
    ) +
            geomHistogram(alpha = .3, binWidth = 1, center = .5) { x = "medianDistanceToVuln"; } +
            xlim(limits = Pair(0, 10)) +
            ggsize(1800, 800)
p

-0.5 
 
 
 
 
 
 
 
 
 0.0 
 
 
 
 
 
 
 
 
 0.5 
 
 
 
 
 
 
 
 
 1.0 
 
 
 
 
 
 
 
 
 1.5 
 
 
 
 
 
 
 
 
 2.0 
 
 
 
 
 
 
 
 
 2.5 
 
 
 
 
 
 
 
 
 3.0 
 
 
 
 
 
 
 
 
 3.5 
 
 
 
 
 
 
 
 
 4.0 
 
 
 
 
 
 
 
 
 4.5 
 
 
 
 
 
 
 
 
 5.0 
 
 
 
 
 
 
 
 
 5.5 
 
 
 
 
 
 
 
 
 6.0 
 
 
 
 
 
 
 
 
 6.5 
 
 
 
 
 
 
 
 
 7.0 
 
 
 
 
 
 
 
 
 7.5 
 
 
 
 
 
 
 
 
 8.0 
 
 
 
 
 
 
 
 
 8.5 
 
 
 
 
 
 
 
 
 9.0 
 
 
 
 
 
 
 
 
 9.5 
 
 
 
 
 
 
 
 
 10.0 
 
 
 
 
 
 
 
 
 10.5 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 700 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 900 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 1,100 
 
 
 
 
 
 
 1,200 
 
 
 
 
 
 
 1,300 
 
 
 
 
 
 
 1,400 
 
 
 
 
 
 
 1,500 
 
 
 
 
 
 
 1,600 
 
 
 
 
 
 
 1,700 
 
 
 
 
 
 
 1,800 
 
 
 
 
 
 
 1,900 
 
 
 
 
 
 
 2,000 
 
 
 
 
 
 
 
 count 
 
 
 
 
 medianDistanceToVuln

In [21]:
val p =
    letsPlot(mapOf(
        "averageDistanceToVuln" to vulnInfo.map { it.averageDistToVuln })
    ) +
            geomHistogram(alpha = .3, binWidth = 1, center = .5) { x = "averageDistanceToVuln"; } +
            xlim(limits = Pair(0, 10)) +
            ggsize(1800, 800)
p

-0.5 
 
 
 
 
 
 
 
 
 0.0 
 
 
 
 
 
 
 
 
 0.5 
 
 
 
 
 
 
 
 
 1.0 
 
 
 
 
 
 
 
 
 1.5 
 
 
 
 
 
 
 
 
 2.0 
 
 
 
 
 
 
 
 
 2.5 
 
 
 
 
 
 
 
 
 3.0 
 
 
 
 
 
 
 
 
 3.5 
 
 
 
 
 
 
 
 
 4.0 
 
 
 
 
 
 
 
 
 4.5 
 
 
 
 
 
 
 
 
 5.0 
 
 
 
 
 
 
 
 
 5.5 
 
 
 
 
 
 
 
 
 6.0 
 
 
 
 
 
 
 
 
 6.5 
 
 
 
 
 
 
 
 
 7.0 
 
 
 
 
 
 
 
 
 7.5 
 
 
 
 
 
 
 
 
 8.0 
 
 
 
 
 
 
 
 
 8.5 
 
 
 
 
 
 
 
 
 9.0 
 
 
 
 
 
 
 
 
 9.5 
 
 
 
 
 
 
 
 
 10.0 
 
 
 
 
 
 
 
 
 10.5 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 1,200 
 
 
 
 
 
 
 1,400 
 
 
 
 
 
 
 1,600 
 
 
 
 
 
 
 1,800 
 
 
 
 
 
 
 2,000 
 
 
 
 
 
 
 2,200 
 
 
 
 
 
 
 2,400 
 
 
 
 
 
 
 2,600 
 
 
 
 
 
 
 
 count 
 
 
 
 
 averageDistanceToVuln

In [22]:
val p =
    letsPlot(
        mapOf("minDistanceToVuln" to vulnInfo.map { it.maxDistToVuln })
    ) { x = "minDistanceToVuln" } + ggsize(1800, 800) +
            geomHistogram(alpha = .3, binWidth = 1, center = .5) +
            geomDensity(alpha = 0, size = 1, adjust = 2) { y = "..count.." } + 
            xlim(limits = Pair(0, 30))

p

<path d="M78.2274585394901 273.4845695020515 L78.2274585394901 273.4845695020515 L81.28919859974411 260.75199491797304 L84.35093865999814 248.5366883426692 L87.41267872025215 236.8700108127369 L90.47441878050617 225.77908197280556 L93.53615884076018 215.28669010035082 L96.59789890101419 205.41126455461244 L99.65963896126821 196.16690876574114 L102.72137902152222 187.56349074648324 L105.78311908177625 179.60678708554838 L108.84485914203026 172.29867549237372 L111.90659920228427 165.6373702286503 L114.96833926253828 159.61769418183053 L118.0300793227923 154.23037974532008 L121.09181938304633 149.46608502533707 L124.15355944330034 145.31057617269244 L127.21529950355435 141.7483016884005 L130.27703956380836 138.76198433541072 L133.3387796240624 136.33296907127738 L136.4005196843164 134.44156114331452 L139.46225974457042 133.06734902345988 L142.52399980482443 132.18950748445673 L145.58573986507844 131.78707679669014 L148.64747992533245 131.83921473105136 L151.70921998558646 132.3254187770342 L154.7709600458405 133.22571670067384 L157.8327001060945 134.52082426692652 L160.89444016634852 136.19226961717243 L163.95618022660256 138.22248441676004 L167.01792028685657 140.59486246206598 L167.01792028685657 140.59486246206598 L170.07966034711058 143.2933197124629 L173.14140040736459 146.30401440584978 L176.2031404676186 149.61290956812513 L179.2648805278726 153.2072221689026 L182.32662058812662 157.07499014528048 L185.38836064838063 161.2049742940319 L188.45010070863466 165.58654342318994 L191.51184076888867 170.20954590038536 L194.57358082914268 175.06417079765845 L197.6353208893967 180.1408018506462 L200.69706094965073 185.4298674264778 L203.75880100990474 190.9216896409912 L206.82054107015875 196.6063356731098 L209.88228113041276 202.47347420676897 L212.94402119066677 208.51223978446365 L216.0057612509208 214.71110768660083 L219.0675013111748 221.05778175611886 L222.12924137142883 227.5388335803658 L225.19098143168284 234.14059506139034 L228.25272149193685 240.84773823696503 L231.3144615521909 247.64408382918344 L234.37620161244487 254.51238149855124 L237.4379416726989 261.4343126408074 L240.49968173295292 268.3905161196767 L243.56142179320693 275.3606372292969 L246.62316185346094 282.32339987345694 L249.68490191371498 289.256701646299 L252.746641973969 296.1377311928037 L255.808382034223 302.9431069278752 L258.870122094477 309.6490359039925 L261.931862154731 316.2314913281499 L264.99360221498506 322.6664069629885 L268.055342275239 328.9298863832797 L268.055342275239 328.9298863832797 L271.1170823354931 334.9984248183004 L274.1788223957471 340.84876872668724 L277.24056245600116 346.4595277109033 L280.30230251625517 351.80949997825877 L283.3640425765092 356.8791162260253 L286.4257826367632 361.650400069377 L289.48752269701714 366.10720585407296 L292.5492627572712 370.2354478844374 L295.6110028175252 374.0233176060234 L298.6727428777792 377.46148527487287 L301.73448293803324 380.54328267105694 L304.79622299828725 383.26486349246534 L307.85796305854126 385.6253381920432 L310.91970311879527 387.62688019966714 L313.9814431790493 389.27480071176694 L317.0431832393033 390.57758952799077 L320.1049232995573 391.54691977718875 L323.1666633598113 392.1976147966215 L326.2284034200654 392.54732958303015 L329.2901434803194 392.6173467609879 L332.3518835405734 392.4311556344554 L335.4136236008274 392.0150536930319 L338.4753636610814 391.3977297650562 L341.5371037213354 390.60999764170236 L344.59884378158944 389.68449581897636 L347.66058384184345 388.6553565957603 L350.7223239020975 387.5578484246073 L353.7840639623515 386.4279960300754 L356.84580402260553 385.30297546870844 L359.9075440828595 384.21734288516427 L362.96928414311355 383.2080014495929 L366.03102420336756 382.3099335936484 L369.0927642636216 381.55683949057556 L372.1545043238756 380.98074284012375 L375.2162443841296 380.61161629281503 L378.2779844443836 380.47696444039224 L381.3397245046377 380.6017571944599 L384.4014645648917 381.00779787646013 L387.46320462514564 381.7137022247883 L390